In [1]:
import pickle
from utils import load_from_folder

dict_dir = "../datasets/logistics/optimal_plans/dictionaries_and_plans"

[action_dict, goals_dict] = load_from_folder(dict_dir, ['dizionario', 'dizionario_goal'])

# print(action_dict.keys())
# for key, value in action_dict.items():
#     print(f"Key: {key}\n\tValue:{value}")

# print(goals_dict)
for key, value in goals_dict.items():
    print(f"Key: {key}\n\tValue:{value}")

dizionario loaded from ../datasets/logistics/optimal_plans/dictionaries_and_plans
dizionario_goal loaded from ../datasets/logistics/optimal_plans/dictionaries_and_plans
Key: at obj33 pos66
	Value:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Key: at obj13 pos13
	Value:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0.

In [3]:
#take organized solution, create a class to represent a plan with initial state, actions and goals
#we have to start by iteratining on the organized solutions we have, and search the corrisponding problem pddl file in the dataset folder to extract initial state and goals
#actions can be taken from the .SOL file

import os
import re

class Plan:
    def __init__(self, problem_id, problem_full_name, initial_state, actions, goals):
        self.problem_id = problem_id
        self.problem_full_name = problem_full_name
        self.initial_state = initial_state
        self.actions = actions
        self.goals = goals

    def __repr__(self):
        return (f"Plan(problem_id={self.problem_id},\n"
                f"     problem_full_name={self.problem_full_name},\n"
                f"     initial_state={self.initial_state},\n"
                f"     actions={self.actions},\n"
                f"     goals={self.goals})")

def parse_pddl(pddl_file_path):
    """
    Parses a PDDL file to extract initial state and goals.
    This simple parser iterates through the file line by line.
    """
    initial_state = []
    goals = []
    mode = None
    try:
        with open(pddl_file_path, 'r') as f:
            for line in f:
                line_stripped = line.strip()
                # Start of initial state block
                if line_stripped.startswith("(:init"):
                    mode = "init"
                    # If there is content on the same line
                    remainder = line_stripped[len("(:init"):].strip()
                    if remainder and remainder != "(" and not remainder.startswith(";"):
                        remainder = remainder.replace("(","").replace(")","")
                        initial_state.append(remainder)
                    continue
                # Start of goal block
                elif line_stripped.startswith("(:goal"):
                    mode = "goal"
                    remainder = line_stripped[len("(:goal"):].strip()
                    # Handle goal starting with (and ... if needed
                    if remainder.startswith("(and"):
                        remainder = remainder[len("(and"):].strip()
                        if remainder and remainder != "(" and not remainder.startswith(";"):
                            remainder = remainder.replace("(","").replace(")","")
                            goals.append(remainder)
                    continue
                # End of a block
                elif line_stripped == ")" or line_stripped == ")))":
                    mode = None
                    continue

                # Append lines based on current block
                if mode == "init":
                    if line_stripped and not line_stripped.startswith(";"):
                        line_stripped = line_stripped.replace("(","").replace(")","")
                        if line_stripped:
                            initial_state.append(line_stripped)
                elif mode == "goal":
                    if line_stripped and not line_stripped.startswith(";"):
                        line_stripped = line_stripped.replace("(","").replace(")","")
                        if line_stripped:
                            goals.append(line_stripped)
    except Exception as e:
        print(f"Error parsing PDDL file {pddl_file_path}: {e}")
        raise Exception
    return initial_state, goals

def parse_sol(sol_file_path):
    """
    Parses a SOL file to extract the list of actions.
    Skips commented lines.
    """
    actions = []
    try:
        with open(sol_file_path, 'r') as f:
            for line in f:
                line_stripped = line.strip()
                # Skip comments (lines starting with ;)
                if line_stripped.startswith(";") or not line_stripped:
                    continue
                line_stripped = line_stripped.replace("(","").replace(")","")
                actions.append(line_stripped)
    except Exception as e:
        print(f"Error parsing SOL file {sol_file_path}: {e}")
    return actions

# Base directories for organized solutions and dataset PDDL files.
# Assumes the script is run from /home/rsignoroni/fastdownward_docker/
solutions_organized_dir = "../datasets/gr_logistics/solutions_organized"
dataset_pddl_base = "../datasets/gr_logistics/problems"  # adjust as needed

plans = []
missing_count=0
# Iterate over each problem directory inside solutions_organized
for problem in os.listdir(solutions_organized_dir):
    problem_dir = os.path.join(solutions_organized_dir, problem)
    if not os.path.isdir(problem_dir):
        continue

    sols_dir = os.path.join(problem_dir, "sols")
    if not os.path.exists(sols_dir):
        print(f"No 'sols' directory for problem {problem} in organized solutions.")
        continue

    # For simplicity, pick the first SOL file we find
    sol_files = [f for f in os.listdir(sols_dir) if f.upper().endswith(".SOL")]
    if not sol_files:
        print(f"No SOL files found in {sols_dir}")
        continue
    temp = []
    print(sol_files)
    for sol_file in sol_files:
        sol_file_path = os.path.join(sols_dir, sol_file)
        actions = parse_sol(sol_file_path)
        version_name = sol_file.split(".")[0]
        # Determine the corresponding PDDL file.
        # Eg: logistics/0-0.2/p000146/p000146_0.pddl
        pddl_file_path = None
        for folder in ["0-0.2", "0.2-0.4", "0.4-0.6", "0.6-0.8", "0.8-1.0"]:
            candidate_path = os.path.join(dataset_pddl_base, folder, problem, f"{version_name}.pddl")
            if os.path.exists(candidate_path):
                pddl_file_path = candidate_path
                break
        if pddl_file_path is None:
            print(f"PDDL file not found for problem {version_name} in any subfolder.")
            missing_count+=1
            continue

        initial_state, goals = parse_pddl(pddl_file_path)

        # Create a Plan object and add to the list.
        temp.append(Plan(problem_id=problem, problem_full_name=version_name, initial_state=initial_state, actions=actions, goals=goals))
        
    if len(temp) == 6:
        plans.extend(temp)
    else:
        print(f"{problem} has missing versions")

# Print out all collected plans
for plan in plans:
    print(plan)
    print("=" * 40)
    
print(len(plans))
print(missing_count)

['p067035_0.SOL', 'p067035_3.SOL', 'p067035_4.SOL', 'p067035_2.SOL', 'p067035_1.SOL', 'p067035_og.SOL']
['p009055_1.SOL', 'p009055_4.SOL', 'p009055_og.SOL', 'p009055_0.SOL', 'p009055_2.SOL', 'p009055_3.SOL']
['p043653_2.SOL', 'p043653_3.SOL', 'p043653_4.SOL', 'p043653_0.SOL', 'p043653_1.SOL']
p043653 has missing versions
['p003058_4.SOL', 'p003058_1.SOL', 'p003058_og.SOL', 'p003058_3.SOL', 'p003058_2.SOL', 'p003058_0.SOL']
['p002938_0.SOL', 'p002938_4.SOL', 'p002938_1.SOL', 'p002938_3.SOL', 'p002938_og.SOL', 'p002938_2.SOL']
['p012799_0.SOL', 'p012799_4.SOL', 'p012799_og.SOL', 'p012799_2.SOL', 'p012799_1.SOL', 'p012799_3.SOL']
['p037417_0.SOL', 'p037417_2.SOL', 'p037417_1.SOL', 'p037417_og.SOL', 'p037417_4.SOL', 'p037417_3.SOL']
['p039490_0.SOL', 'p039490_4.SOL', 'p039490_2.SOL', 'p039490_3.SOL', 'p039490_1.SOL', 'p039490_og.SOL']
['p044307_3.SOL', 'p044307_0.SOL', 'p044307_4.SOL', 'p044307_1.SOL', 'p044307_2.SOL']
p044307 has missing versions
['p002054_og.SOL', 'p002054_2.SOL', 'p0020

In [ ]:
#create action and goal dictionaries
import pickle
unique_actions = set()
for plan in plans:
    unique_actions.update(plan.actions)
        
unique_actions = list(unique_actions)

print(len(unique_actions))

action_dict = {}

for i, action in enumerate(unique_actions):
    action_dict[action] = i
    
print(action_dict)


unique_goals = set()
for plan in plans:
    unique_goals.update(plan.goals)

# Sort to have a consistent ordering
unique_goals = sorted(unique_goals)
num_goals = len(unique_goals)
goal_dict = {}

# Build the one-hot encoding for each goal
for idx, goal in enumerate(unique_goals):
    onehot = [0] * num_goals
    onehot[idx] = 1
    goal_dict[goal] = onehot

# Print the goal dictionary
print(goal_dict)

with(open("../datasets/gr_logistics/pickles/action_dict.pkl", "wb")) as ad:
    pickle.dump(action_dict, ad)
    
with(open("../datasets/gr_logistics/pickles/goal_dict.pkl", "wb")) as gd:
    pickle.dump(goal_dict, gd)

7704
{'unload-truck obj33 tru5 pos55': 0, 'drive-truck tru3 apt4 pos22 cit4': 1, 'unload-airplane obj33 apn7 apt3': 2, 'drive-truck tru5 apt5 pos55 cit6': 3, 'drive-truck tru3 apt3 pos13 cit3': 4, 'drive-truck tru4 pos22 pos23 cit1': 5, 'unload-truck obj23 tru3 apt2': 6, 'load-airplane obj88 apn2 apt5': 7, 'drive-truck tru4 pos33 apt2 cit6': 8, 'drive-truck tru1 apt7 pos33 cit3': 9, 'drive-truck tru1 pos22 pos55 cit3': 10, 'drive-truck tru5 pos12 apt5 cit4': 11, 'load-truck obj88 tru5 pos44': 12, 'drive-truck tru2 pos11 apt7 cit2': 13, 'load-truck obj77 tru5 pos11': 14, 'unload-truck obj77 tru5 apt7': 15, 'unload-airplane obj44 apn7 apt6': 16, 'drive-truck tru3 pos21 pos77 cit2': 17, 'load-truck obj88 tru2 pos44': 18, 'load-airplane obj13 apn7 apt7': 19, 'unload-airplane obj44 apn7 apt1': 20, 'unload-truck obj23 tru5 pos55': 21, 'unload-truck obj77 tru3 pos33': 22, 'drive-truck tru5 apt2 pos11 cit5': 23, 'unload-truck obj66 tru4 apt7': 24, 'unload-truck obj00 tru3 pos44': 25, 'load-tru

In [ ]:
import random

#now need to pickle train, val, test plans
# Shuffle the plans list to randomize the order
random.shuffle(plans)
n = len(plans)
train_end = int(n * 0.8)
val_end = train_end + int(n * 0.10)

train_plans = plans[:train_end]
val_plans = plans[train_end:val_end]
test_plans = plans[val_end:]

print(len(train_plans))
print(len(val_plans))
print(len(test_plans))

with open("../datasets/gr_logistics/pickles/train_plans.pkl", "wb") as f:
    pickle.dump(train_plans, f)

with open("../datasets/gr_logistics/pickles/val_plans.pkl", "wb") as f:
    pickle.dump(val_plans, f)

with open("../datasets/gr_logistics/pickles/test_plans.pkl", "wb") as f:
    pickle.dump(test_plans, f)
    


1867
233
234
